# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.mode.chained_assignment = None

# Load data

In [ ]:
index_cols = ['year', 'sex', 'age_band', 'geo_code']

ew = pd.read_csv('../projections/combined/england_wales_final.csv').set_index(index_cols)
sc = pd.read_csv('../projections/combined/scotland_final.csv').set_index(index_cols)
ni = pd.read_csv('../projections/combined/northern_ireland_final.csv').set_index(index_cols)

# Country-specific tables

## England & Wales

In [4]:
ew.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Buddhist,Christian,Hindu,Jewish,Muslim,No religion,Not answered,Other religion,Sikh
year,,,,,,,,,
2001,0.3%,71.7%,1.1%,0.5%,3.0%,14.8%,7.7%,0.3%,0.6%
2011,0.4%,59.3%,1.5%,0.5%,4.8%,25.1%,7.2%,0.4%,0.8%
2021,0.5%,46.2%,1.7%,0.5%,6.5%,37.2%,6.0%,0.6%,0.9%
2022,0.5%,44.8%,1.7%,0.5%,6.6%,38.5%,5.9%,0.6%,0.9%
2023,0.5%,43.2%,1.8%,0.5%,6.8%,39.9%,5.8%,0.6%,0.9%
2024,0.5%,41.8%,1.8%,0.5%,7.0%,41.3%,5.7%,0.6%,0.9%
2025,0.5%,40.3%,1.8%,0.5%,7.1%,42.6%,5.6%,0.7%,0.9%
2026,0.5%,38.9%,1.8%,0.5%,7.3%,43.9%,5.5%,0.7%,0.9%
2027,0.5%,37.7%,1.9%,0.5%,7.4%,45.1%,5.4%,0.7%,0.9%


## Scotland

In [5]:
sc.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Buddhist,Church of Scotland,Hindu,Jewish,Muslim,No religion,Other Christian,Other religion,Religion not stated,Roman Catholic,Sikh
year,,,,,,,,,,,
2011,0.2%,32.4%,0.3%,0.1%,1.4%,36.7%,5.5%,0.3%,7.0%,15.9%,0.2%
2021,0.3%,20.4%,0.6%,0.1%,2.2%,51.1%,5.1%,0.6%,6.2%,13.3%,0.2%
2022,0.3%,19.9%,0.6%,0.1%,2.3%,51.7%,5.1%,0.6%,6.1%,13.2%,0.2%
2023,0.3%,18.7%,0.6%,0.1%,2.3%,53.2%,5.0%,0.6%,6.0%,12.9%,0.2%
2024,0.3%,17.6%,0.6%,0.1%,2.4%,54.6%,4.9%,0.6%,5.9%,12.6%,0.2%
2025,0.3%,16.6%,0.6%,0.1%,2.5%,56.0%,4.9%,0.7%,5.8%,12.3%,0.2%
2026,0.3%,15.6%,0.6%,0.1%,2.6%,57.4%,4.8%,0.7%,5.7%,12.0%,0.2%
2027,0.3%,14.6%,0.7%,0.1%,2.6%,58.6%,4.8%,0.7%,5.6%,11.7%,0.2%
2028,0.3%,13.7%,0.7%,0.1%,2.7%,59.8%,4.7%,0.8%,5.5%,11.4%,0.2%


## Northern Ireland

In [6]:
ni.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Catholic,Church of Ireland,Methodist,No religion,Not stated,Other Christian,Other religions,Presbyterian
year,,,,,,,,
2011,40.8%,13.7%,3.0%,10.1%,6.8%,5.8%,0.8%,19.1%
2021,42.3%,11.5%,2.4%,17.4%,1.6%,6.8%,1.3%,16.6%
2022,42.3%,11.3%,2.3%,18.2%,1.3%,6.9%,1.4%,16.3%
2023,42.2%,11.1%,2.2%,19.0%,1.1%,7.0%,1.4%,16.0%
2024,42.1%,10.8%,2.2%,19.8%,0.9%,7.0%,1.5%,15.7%
2025,41.9%,10.6%,2.1%,20.7%,0.7%,7.1%,1.5%,15.4%
2026,41.7%,10.3%,2.0%,21.5%,0.6%,7.1%,1.6%,15.1%
2027,41.6%,10.1%,2.0%,22.2%,0.5%,7.2%,1.6%,14.8%
2028,41.4%,9.8%,1.9%,22.9%,0.5%,7.3%,1.7%,14.5%


# Map to top-level categories

## England & Wales

In [7]:
ew_cat1 = pd.DataFrame()
ew_cat1['Christian'] = ew['Christian']
ew_cat1['No religion'] = ew['No religion']
ew_cat1['Not answered'] = ew['Not answered']
ew_cat1['Other religions'] = ew.iloc[:, [0,2,3,4,7,8]].sum(axis='columns')

In [8]:
ew_cat1.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Christian,No religion,Not answered,Other religions
year,,,,
2001,71.7%,14.8%,7.7%,5.7%
2011,59.3%,25.1%,7.2%,8.4%
2021,46.2%,37.2%,6.0%,10.6%
2022,44.8%,38.5%,5.9%,10.8%
2023,43.2%,39.9%,5.8%,11.0%
2024,41.8%,41.3%,5.7%,11.2%
2025,40.3%,42.6%,5.6%,11.4%
2026,38.9%,43.9%,5.5%,11.6%
2027,37.7%,45.1%,5.4%,11.8%


## Scotland

In [9]:
sc_cat1 = pd.DataFrame()
sc_cat1['Christian'] = sc.loc[:, ['Church of Scotland', 'Other Christian', 'Roman Catholic']].sum(axis='columns')
sc_cat1['No religion'] = sc['No religion']
sc_cat1['Not answered'] = sc['Religion not stated']
sc_cat1['Other religions'] = sc.iloc[:, [0,2,3,4,7,10]].sum(axis='columns')

In [10]:
sc_cat1.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Christian,No religion,Not answered,Other religions
year,,,,
2011,53.8%,36.7%,7.0%,2.6%
2021,38.8%,51.1%,6.2%,3.9%
2022,38.2%,51.7%,6.1%,4.0%
2023,36.7%,53.2%,6.0%,4.1%
2024,35.2%,54.6%,5.9%,4.2%
2025,33.8%,56.0%,5.8%,4.4%
2026,32.4%,57.4%,5.7%,4.5%
2027,31.1%,58.6%,5.6%,4.7%
2028,29.8%,59.8%,5.5%,4.8%


In [11]:
sc_cat2 = pd.DataFrame()
sc_cat2['Catholic'] = sc['Roman Catholic']
sc_cat2['Protestant'] = sc[['Church of Scotland', 'Other Christian']].sum(axis='columns')
sc_cat2['No religion'] = sc['No religion']
sc_cat2['Not answered'] = sc['Religion not stated']
sc_cat2['Other religions'] = sc.iloc[:, [0,2,3,4,7,10]].sum(axis='columns')

In [12]:
sc_cat2.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Catholic,Protestant,No religion,Not answered,Other religions
year,,,,,
2011,15.9%,37.9%,36.7%,7.0%,2.6%
2021,13.3%,25.5%,51.1%,6.2%,3.9%
2022,13.2%,25.0%,51.7%,6.1%,4.0%
2023,12.9%,23.7%,53.2%,6.0%,4.1%
2024,12.6%,22.6%,54.6%,5.9%,4.2%
2025,12.3%,21.5%,56.0%,5.8%,4.4%
2026,12.0%,20.4%,57.4%,5.7%,4.5%
2027,11.7%,19.4%,58.6%,5.6%,4.7%
2028,11.4%,18.4%,59.8%,5.5%,4.8%


## Northern Ireland

In [13]:
ni_cat1 = pd.DataFrame()
ni_cat1['Christian'] = ni.loc[:, ['Catholic', 'Church of Ireland', 'Methodist', 'Other Christian', 'Presbyterian']].sum(axis='columns')
ni_cat1['No religion'] = ni['No religion']
ni_cat1['Not answered'] = ni['Not stated']
ni_cat1['Other religions'] = ni['Other religions']

In [14]:
ni_cat1.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Christian,No religion,Not answered,Other religions
year,,,,
2011,82.3%,10.1%,6.8%,0.8%
2021,79.7%,17.4%,1.6%,1.3%
2022,79.1%,18.2%,1.3%,1.4%
2023,78.5%,19.0%,1.1%,1.4%
2024,77.8%,19.8%,0.9%,1.5%
2025,77.1%,20.7%,0.7%,1.5%
2026,76.3%,21.5%,0.6%,1.6%
2027,75.6%,22.2%,0.5%,1.6%
2028,74.9%,22.9%,0.5%,1.7%


In [15]:
ni_cat2 = pd.DataFrame()
ni_cat2['Catholic'] = ni['Catholic']
ni_cat2['Protestant'] = ni.loc[:, ['Church of Ireland', 'Methodist', 'Other Christian', 'Presbyterian']].sum(axis='columns')
ni_cat2['No religion'] = ni['No religion']
ni_cat2['Not answered'] = ni['Not stated']
ni_cat2['Other religions'] = ni['Other religions']

In [16]:
ni_cat2.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Catholic,Protestant,No religion,Not answered,Other religions
year,,,,,
2011,40.8%,41.6%,10.1%,6.8%,0.8%
2021,42.3%,37.4%,17.4%,1.6%,1.3%
2022,42.3%,36.9%,18.2%,1.3%,1.4%
2023,42.2%,36.3%,19.0%,1.1%,1.4%
2024,42.1%,35.8%,19.8%,0.9%,1.5%
2025,41.9%,35.2%,20.7%,0.7%,1.5%
2026,41.7%,34.6%,21.5%,0.6%,1.6%
2027,41.6%,34.1%,22.2%,0.5%,1.6%
2028,41.4%,33.5%,22.9%,0.5%,1.7%


# UK Combined

In [17]:
uk_cat1 = pd.concat([ew_cat1, sc_cat1, ni_cat1])

In [18]:
uk_cat1.shape

(190916, 4)

In [19]:
uk_cat1.groupby('year').sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

,Christian,No religion,Not answered,Other religions
year,,,,
2001,71.7%,14.8%,7.7%,5.7%
2011,59.5%,25.7%,7.2%,7.7%
2021,46.5%,37.8%,5.9%,9.8%
2022,45.2%,39.0%,5.8%,10.0%
2023,43.7%,40.4%,5.7%,10.2%
2024,42.3%,41.8%,5.6%,10.4%
2025,40.8%,43.1%,5.5%,10.6%
2026,39.4%,44.4%,5.4%,10.8%
2027,38.2%,45.5%,5.3%,11.0%


# Add generation

In [20]:
uk_cat1_gen = uk_cat1.reset_index()

uk_cat1_gen['end_yob'] = uk_cat1_gen['year'] - uk_cat1_gen['age_band'].str.split('-').str[0].str[:2].astype(int)

gen_bounds = [0, 1946, 1967, 1982, 1997, 2012, 2100]
gen_labels = ['silent', 'boomers', 'genX', 'millennials', 'genZ', 'alpha']

uk_cat1_gen['generation'] = pd.cut(uk_cat1_gen['end_yob'], bins=gen_bounds, labels=gen_labels)

uk_cat1_gen = uk_cat1_gen.drop(columns=['age_band','end_yob']).set_index(['year', 'sex', 'generation', 'geo_code'])

In [21]:
# uk_cat1_gen.groupby(['generation', 'year'], observed=True).sum().apply(lambda x: x/x.sum(), axis='columns').style.format('{:.1%}')

# Add country

In [22]:
uk_cat1 = uk_cat1.reset_index()
uk_cat1['country'] = uk_cat1['geo_code'].str[0].map({'E': 'England', 'W': 'Wales', 'S': 'Scotland', '9': 'Northern Ireland'})

In [23]:
cat1_cols = ['Christian','No religion','Not answered','Other religions']

# Datawrapper charts

## Overall share progression line chart

In [24]:
(uk_cat1.groupby('year')[cat1_cols].sum().apply(lambda x: x/x.sum(), axis='columns') * 100).to_csv('line_chart.csv')

## Arrow plot

In [25]:
gen_map = {
    'silent': 'Silent Generation',
    'boomers': 'Baby Boomers',
    'genX': 'Generation X',
    'millennials': 'Millennials',
    'genZ': 'Generation Z'
}

In [26]:
(uk_cat1_gen.groupby(['year', 'generation', 'sex'], observed=True)[cat1_cols].sum()
 .apply(lambda x: x/x.sum(), axis=1)['No religion']
 .unstack('year')[[2011, 2021, 2031]] * 100).reset_index().replace({'generation': gen_map}).to_csv('arrow_plot.csv', index=False)

/var/folders/ck/yxzr5lpj1896cp_ybtd8xv640000gn/T/ipykernel_66839/1543367859.py:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  .unstack('year')[[2011, 2021, 2031]] * 100).reset_index().replace({'generation': gen_map}).to_csv('arrow_plot.csv', index=False)


## Year of birth cohort dot plot

In [27]:
uk_cat1['end_yob'] = uk_cat1['year'] - uk_cat1['age_band'].str.split('-').str[0].str[:2].astype(int)
uk_cat1['yob_band'] = np.where(uk_cat1['age_band'].str[2] == '+', 'pre-' + uk_cat1['end_yob'].astype(str), (uk_cat1['end_yob'] - 4).astype(str).str.cat(uk_cat1['end_yob'].astype(str), '-'))
uk_cat1 = uk_cat1.drop(columns=['end_yob'])

In [28]:
uk_cat1['yob_band'] = np.where(uk_cat1['yob_band'].str[-4:].astype(int) < 1947, 'pre-1947', uk_cat1['yob_band'])

In [29]:
uk_cat1['yob_band'] = pd.Categorical(
    uk_cat1['yob_band'],
    categories=sorted(uk_cat1['yob_band'].unique(), key=lambda x: x[-4:])
)

In [30]:
for_yob_band_plot = uk_cat1.loc[uk_cat1['year'].isin([2011,2021,2031])].groupby(['year', 'yob_band'], observed=True)[cat1_cols].sum().apply(lambda x: x/x.sum(), axis=1)['No religion'].unstack('year') * 100

In [31]:
for_yob_band_plot.to_csv('cohort_dotplot.csv')

## Chloropleth

In [32]:
out = uk_cat1.groupby(['year', 'geo_code'])[cat1_cols].sum().apply(lambda x: x/x.sum(), axis='columns').loc[[2011, 2021, 2025, 2031]].unstack('year')
out.columns = [f'{i[0]}_{i[1]}' for i in out.columns.to_flat_index()]
# out

In [33]:
out_pct = out * 100

In [34]:
geo_lookup = pd.read_csv('geo_lookup.csv', index_col='geo_code')

In [35]:
out_pct.join(geo_lookup).to_csv('chloropleth.csv')